# 613 - Pythagorean Ant

## Problem Statement

<p>Dave is doing his homework on the balcony and, preparing a presentation about Pythagorean triangles, has just cut out a triangle with side lengths 30cm, 40cm and 50cm from some cardboard, when a gust of wind blows the triangle down into the garden.<br />
Another gust blows a small ant straight onto this triangle. The poor ant is completely disoriented and starts to crawl straight ahead in random direction in order to get back into the grass.</p>

<p>Assuming that all possible positions of the ant within the triangle and all possible directions of moving on are equiprobable, what is the probability that the ant leaves the triangle along its longest side?<br />
Give your answer rounded to 10 digits after the decimal point.</p> 

## Solution

Without loss of generality, let's assume that the right angle if at (0, 0), the side of length 40 is on the x-axis and the remaining side is on the y-axis. When the ant is at a point $(x, y)$, it will leave the triangle along the hypothenuse only if its direction is within the angle subtended by the hypothenuse. Using the law of cosine, this angle is given by

\begin{equation}
    \theta = \arccos \left( \frac{A^2 + B^2 - 50^2}{2AB} \right)
\end{equation}

where $A$ and $B$ are simply the distance between the point $(x, y)$ and the vertices of the triangle given by

\begin{equation}
    A = \sqrt{(x - 40)^2 + y^2}
\end{equation}

and 

\begin{equation}
    B = \sqrt{x^2 + (y - 30)^2}.
\end{equation}

In order to obtain a probability, we divide by $2\pi$ (the angle of a full circle).

In order to integrate over the region, we need to determine the joint PDF of $x$ and $y$ and their bounds of integration. All the $(x, y)$ points are equally likely so the PDF is simply $\frac{1}{600}$ (600 is the area of the triangle). The bound of integration for $x$ is from 0 to 40. The upper bound of integration for $y$ is the slope of the hypothenuse which is equal $-\frac{3}{4}x + 30$ and the lower bound is 0. The probability, $p$, that the ant leaves along the hypotenuse is therefore given by

\begin{equation}
    p = \frac{1}{1200\pi} \int_{0}^{40} \int_0^{-\frac{3}{4}x + 30} \arccos \left( \frac{A^2 + B^2 - 50^2}{2AB} \right) dydx.
\end{equation}

We will proceed with numerical integration.

In [2]:
from scipy.integrate import dblquad
import numpy as np

# Define the function to compute the angle subtended by the hypotenuse
def subtended_angle(x, y):
    d1 = np.sqrt(x**2 + y**2)  # Distance to (40, 0)
    d2 = np.sqrt((x - 40)**2 + (y - 30)**2)  # Distance to (0, 30)
    hypotenuse = 50  # Length of the hypotenuse
    # Calculate the cosine of the angle using the Law of Cosines
    cos_theta = (d1**2 + d2**2 - hypotenuse**2) / (2 * d1 * d2)
    # Clip the cosine to the interval [-1, 1] to avoid errors in arccos
    cos_theta = np.clip(cos_theta, -1, 1)
    angle = np.arccos(cos_theta)
    return angle / (2 * np.pi)  # Normalize by 2*pi

# Define the limits for y, as functions of x
def y_lower(x):
    return 0

def y_upper(x):
    return -3/4 * x + 30

area, error = dblquad(
    lambda y, x: subtended_angle(x, y) * 1/600,
    0, 40,
    y_lower, y_upper,
    epsabs=1e-18,
    epsrel=1e-18,
)

print(f"Probability: {area:.10f}")
print(f"Estimation error: {error}")

Probability: 0.3916721504
Estimation error: 9.11188838381975e-08
